<a href="https://colab.research.google.com/github/yewonize/ssu-soccer-stats/blob/main/%EC%88%AD%EC%8B%A4%EB%8C%80%ED%95%99%EA%B5%90_%EC%B6%95%EA%B5%AC%EB%8B%A8_%EA%B8%B0%EB%A1%9D%EC%8B%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd

# -----------------------------------------------------------------------------
# 1. 페이지 기본 설정
# -----------------------------------------------------------------------------
st.set_page_config(
    page_title="숭실대학교 축구단 기록실",
    page_icon="⚽",
    layout="wide"
)

# -----------------------------------------------------------------------------
# 2. 데이터 로드 함수 (캐싱 적용)
# -----------------------------------------------------------------------------
@st.cache_data
def load_data():
    # 파일 경로 설정 (같은 폴더에 위치해야 함)
    file_player = "숭실대학교 축구단 기록 - 경기별선수기록.csv"
    file_match = "숭실대학교 축구단 기록 - 경기기록.csv"

    try:
        df_p = pd.read_csv(file_player)
        df_m = pd.read_csv(file_match)

        # 날짜 컬럼을 문자열로 통일 (혹시 모를 오류 방지)
        df_p['날짜'] = df_p['날짜'].astype(str)
        df_m['날짜'] = df_m['날짜'].astype(str)

        return df_p, df_m
    except FileNotFoundError:
        return None, None

df_player, df_match = load_data()

# 데이터 로드 실패 시 처리
if df_player is None or df_match is None:
    st.error("❌ 데이터 파일을 찾을 수 없습니다. '숭실대학교 축구단 기록 - 경기별선수기록.csv'와 '숭실대학교 축구단 기록 - 경기기록.csv' 파일이 같은 폴더에 있는지 확인해주세요.")
    st.stop()

# -----------------------------------------------------------------------------
# 3. 사이드바: Top-down 방식 필터 (터치다운 필터)
# -----------------------------------------------------------------------------
st.sidebar.header("🔍 검색 조건 (Filter)")
st.sidebar.markdown("위에서부터 순서대로 선택하면\n하위 목록이 자동으로 갱신됩니다.")

# 필터링을 위한 임시 데이터프레임 (단계별로 줄여나감)
current_df = df_player.copy()

# (1) 연도 선택
all_years = sorted(current_df['연도'].unique(), reverse=True)
selected_years = st.sidebar.multiselect("1️⃣ 연도 (Year)", all_years, default=all_years)

if selected_years:
    current_df = current_df[current_df['연도'].isin(selected_years)]

# (2) 경기일자 선택 (선택된 연도에 해당하는 날짜만 노출)
available_dates = sorted(current_df['날짜'].unique(), reverse=True)
selected_dates = st.sidebar.multiselect("2️⃣ 경기일자 (Date)", available_dates)

if selected_dates:
    current_df = current_df[current_df['날짜'].isin(selected_dates)]

# (3) 대회/리그 선택 (선택된 날짜/연도에 해당하는 대회만 노출)
available_tournaments = sorted(current_df['대회명'].unique())
selected_tournaments = st.sidebar.multiselect("3️⃣ 대회/리그 (Tournament)", available_tournaments)

if selected_tournaments:
    current_df = current_df[current_df['대회명'].isin(selected_tournaments)]

# (4) 상대교 선택
available_opponents = sorted(current_df['상대팀'].unique())
selected_opponents = st.sidebar.multiselect("4️⃣ 상대교 (Opponent)", available_opponents)

if selected_opponents:
    current_df = current_df[current_df['상대팀'].isin(selected_opponents)]

# (5) 선수명 선택
available_players = sorted(current_df['선수명'].unique())
selected_players = st.sidebar.multiselect("5️⃣ 선수명 (Player)", available_players)

# 최종 선수 데이터 확정
final_player_df = current_df.copy()
if selected_players:
    final_player_df = final_player_df[final_player_df['선수명'].isin(selected_players)]

# -----------------------------------------------------------------------------
# 4. 경기 기록 데이터 매칭 (Filtering Match Records)
# -----------------------------------------------------------------------------
# 최종 선택된 선수 기록에 등장하는 '날짜'와 '상대팀'을 기준으로 경기 기록을 가져옵니다.
relevant_matches = final_player_df[['날짜', '상대팀']].drop_duplicates()
final_match_df = df_match.merge(relevant_matches, on=['날짜', '상대팀'], how='inner')

# -----------------------------------------------------------------------------
# 5. 메인 화면 구성
# -----------------------------------------------------------------------------
st.title("⚽ 숭실대학교 축구단 기록실")
st.markdown("선택한 조건에 맞는 **경기 기록**과 **선수 기록**을 보여줍니다.")

# 간단한 대시보드 (통계)
col1, col2, col3 = st.columns(3)
with col1:
    st.metric("검색된 경기 수", f"{len(final_match_df)} 경기")
with col2:
    total_goals = final_player_df['득점'].sum() if not final_player_df.empty else 0
    st.metric("선수 득점 합계 (검색된 선수)", f"{int(total_goals)} 골")
with col3:
    if not final_player_df.empty:
        mom_count = len(final_player_df[final_player_df['MOM'] == 1])
        st.metric("MOM 선정 횟수", f"{mom_count} 회")
    else:
        st.metric("MOM 선정 횟수", "0 회")

st.divider()

# 탭 구성: 선수 기록 / 경기 기록
tab1, tab2 = st.tabs(["🏃 선수별 세부 기록", "📅 경기 결과 요약"])

# [Tab 1] 선수 기록 테이블
with tab1:
    st.subheader("선수별 세부 기록")
    if not final_player_df.empty:
        # 보여줄 컬럼 순서 정리
        p_cols = ['연도', '대회명', '라운드', '날짜', '상대팀', '선수명', '선발/교체', '출전시간', '득점', '도움', 'MOM', '경고', '비고']
        # 실제 데이터프레임에 존재하는 컬럼만 선택
        valid_p_cols = [c for c in p_cols if c in final_player_df.columns]

        display_p_df = final_player_df[valid_p_cols].copy()

        # MOM 데이터 시각화 (1 -> ⭐)
        if 'MOM' in display_p_df.columns:
            display_p_df['MOM'] = display_p_df['MOM'].apply(lambda x: '⭐' if x == 1 else '')

        st.dataframe(
            display_p_df,
            use_container_width=True,
            hide_index=True,
            column_config={
                "출전시간": st.column_config.NumberColumn(format="%d분"),
                "득점": st.column_config.NumberColumn(format="%d골"),
                "도움": st.column_config.NumberColumn(format="%d도움"),
            }
        )
    else:
        st.info("조건에 맞는 선수 기록이 없습니다. 필터를 조정해 보세요.")

# [Tab 2] 경기 기록 테이블
with tab2:
    st.subheader("경기 결과 요약")
    if not final_match_df.empty:
        # 보여줄 컬럼
        m_cols = ['연도', '대회명', '라운드', '날짜', '상대팀', '스코어', '득점자', '비고']
        valid_m_cols = [c for c in m_cols if c in final_match_df.columns]

        st.dataframe(
            final_match_df[valid_m_cols],
            use_container_width=True,
            hide_index=True
        )
    else:
        st.info("조건에 맞는 경기 기록이 없습니다.")